In [28]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)

import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer as ps
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package genesis to
[nltk_data]     C:\Users\ASUSµ\AppData\Roaming\nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUSµ\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUSµ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUSµ\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [29]:
a= pd.read_csv('allDataWithLabel.txt')

In [35]:
a.head(20)
data = pd.DataFrame(a.article)
data['label']=a['label']
data.head()

,article,label
0,Mosaïque FM a appris que le président du Parle...,autres
1,Mosaïque FM a appris que les présidents des bl...,autres
2,Nidaa Tounes a annoncé dans un communiqué rend...,nidaa
3,L’instance de contrôle de la constitutionnalit...,autres
4,"Le parti tunisien pour la paix sociale, créé o...",autres


In [36]:
class KNN_NLC_Classifer():
    def __init__(self, k=1, distance_type = 'path'):
        self.k = k
        self.distance_type = distance_type

    # This function is used for training
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    # This function runs the K(1) nearest neighbour algorithm and
    # returns the label with closest match. 
    def predict(self, x_test):
        self.x_test = x_test
        y_predict = []

        for i in range(len(x_test)):
            max_sim = 0
            max_index = 0
            for j in range(self.x_train.shape[0]):
                temp = self.document_similarity(x_test[i], self.x_train[j])
                if temp > max_sim:
                    max_sim = temp
                    max_index = j
            y_predict.append(self.y_train[max_index])
        return y_predict
            
    def convert_tag(self, tag):
        """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
        tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
        try:
            return tag_dict[tag[0]]
        except KeyError:
            return None


    def doc_to_synsets(self, doc):
        """
                Returns a list of synsets in document.
                Tokenizes and tags the words in the document doc.
                Then finds the first synset for each word/tag combination.
            If a synset is not found for that combination it is skipped.

            Args:
                doc: string to be converted

            Returns:
                list of synsets
        """
        tokens = word_tokenize(doc+' ')  
        l = []
        tags = nltk.pos_tag([tokens[0] + ' ']) if len(tokens) == 1 else nltk.pos_tag(tokens)

        for token, tag in zip(tokens, tags):
            syntag = self.convert_tag(tag[1])
            syns = wn.synsets(token, syntag)
            if (len(syns) > 0):
                l.append(syns[0])
        return l  
    
    def similarity_score(self, s1, s2, distance_type = 'path'):
        """
          Calculate the normalized similarity score of s1 onto s2
          For each synset in s1, finds the synset in s2 with the largest similarity value.
          Sum of all of the largest similarity values and normalize this value by dividing it by the
          number of largest similarity values found.

          Args:
              s1, s2: list of synsets from doc_to_synsets

          Returns:
              normalized similarity score of s1 onto s2
        """
        s1_largest_scores = []

        for i, s1_synset in enumerate(s1, 0):
            max_score = 0
            for s2_synset in s2:
                if distance_type == 'path':
                    score = s1_synset.path_similarity(s2_synset, simulate_root = False)
                else:
                    score = s1_synset.wup_similarity(s2_synset)                  
                if score != None:
                    if score > max_score:
                        max_score = score
              
            if max_score != 0:
                s1_largest_scores.append(max_score)
          
        mean_score = np.mean(s1_largest_scores)
                 
        return mean_score
    
    def document_similarity(self,doc1, doc2):
        """Finds the symmetrical similarity between doc1 and doc2"""

        synsets1 = self.doc_to_synsets(doc1)
        synsets2 = self.doc_to_synsets(doc2)

        return (self.similarity_score(synsets1, synsets2) + self.similarity_score(synsets2, synsets1)) / 2

    
    

In [53]:
output=[]
for label in data['label']:
    if label=='autres':
        output.append(3)
    elif label=='nidaa':
        output.append(2)
    else:
        output.append(1)
        
data['output']=output
Num_Words = data.shape[0]
print("\nSize of input file is ", data.shape)
data.head(30)


Size of input file is  (2106, 3)


,article,label,output
0,Mosaïque FM a appris que le président du Parle...,autres,3
1,Mosaïque FM a appris que les présidents des bl...,autres,3
2,Nidaa Tounes a annoncé dans un communiqué rend...,nidaa,2
3,L’instance de contrôle de la constitutionnalit...,autres,3
4,"Le parti tunisien pour la paix sociale, créé o...",autres,3
5,"Le représentant légal de Nidaa Tounes, Hafedh ...",nidaa,2
6,Le bureau politique de Nidaa Tounes a déclaré ...,nidaa,2
7,Le membre du bureau de l'Assemblée des représe...,autres,3
8,Nidaa Tounes a proposé au président de la Répu...,nidaa,2
9,Le secrétaire général du Parti Tahya Tounes (V...,autres,3


In [57]:
import re
nltk.download('stopwords')
s = stopwords.words('french')
ps = nltk.wordnet.WordNetLemmatizer()
for i in range(data.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', data.loc[i,'article'])
    review = review.lower()
    review = review.split()

    review = [ps.lemmatize(word) for word in review if not word in s]
    review = ' '.join(review)
    data.loc[i, 'article'] = review

X_train = data['article']
y_train = data['output']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUSµ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
final_test_list = data['article'][0:20]
final_test_list[0]

'mosa fm a appris pr sident parlement mohamed naceur a interrompu cong maladie afin rendre assembl e repr sentants peuple tait effet absent depuis quelques jours raison malaise attendu mohamed naceur pr side r union pr sidents bloc parlementaires'

In [65]:
classifier = KNN_NLC_Classifer(k=1, distance_type='path')
classifier.fit(X_train, y_train)



In [ ]:
final_test_list = data['article'][0:50]
                 
test_corpus = []
for i in range(len(final_test_list)):
    review = re.sub('[^a-zA-Z]', ' ', final_test_list[i])
    review = review.lower()
    review = review.split()

    review = [ps.lemmatize(word) for word in review if not word in s]
    review = ' '.join(review)
    test_corpus.append(review)

print('predicting ...')
y_pred_final = classifier.predict(test_corpus)
print('c bn ')
output_df = pd.DataFrame(data = {'article': final_test_list, 'code': y_pred_final})
output_df['label'] = np.where(output_df['code']==1, 'Nahdha','autres')
print(output_df)

predicting ...
